In [1]:
# GRU

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout, Flatten
from tensorflow.keras import optimizers
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from tensorflow.keras.callbacks import EarlyStopping

# Load dataset
file_path = 'data/basic.csv'
data = pd.read_csv(file_path)

# Remove unnecessary columns
data = data.drop(columns=['last_name, first_name'])

# Handle missing values (e.g., replace with 0)
data = data.fillna(0)

# Extract player_id that exists in 2021, 2022, and 2023

data_2021 = data[data['year'] == 2021]
data_2022 = data[data['year'] == 2022]
data_2023 = data[data['year'] == 2023]

player_ids_2021 = set(data_2021['player_id'].unique())
player_ids_2022 = set(data_2022['player_id'].unique())
player_ids_2023 = set(data_2023['player_id'].unique())

common_player_ids = player_ids_2021 & player_ids_2022 & player_ids_2023

# Extract data corresponding to common player_id
common_data = data[data['player_id'].isin(common_player_ids)]

# Extract data corresponding to 2021 and 2022
final = common_data[common_data['year'].isin([2021, 2022])]
final = final.sort_values(by=['player_id', 'year'])

# Select necessary columns (excluding year)
features = [col for col in final.columns if col not in ['player_id', 'year', 'p_era']]
target = 'p_era'

# Separate independent and dependent variables
X = final[features].values
y = final[target].values

# Data scaling
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

# Convert to time series data format
def create_sequences(X, y, seq_length):
    X_seq, y_seq = [], []
    for i in range(len(X)):
        seq_x = X[max(0, i - seq_length + 1):i + 1]
        seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
        seq_y = y[i]
        X_seq.append(seq_x)
        y_seq.append(seq_y)
    return np.array(X_seq), np.array(y_seq)

seq_length = 2  # Set sequence length
X_seq, y_seq = create_sequences(X_scaled, y_scaled, seq_length)

# Set training data same as entire data
X_train, y_train = X_seq, y_seq

# Initialize lists to store metrics
iterations = 5
rmse_list = []
mae_list = []
mape_list = []

for i in range(iterations):
    # Define GRU model
    model_GRU = Sequential()
    model_GRU.add(GRU(64, input_shape=(seq_length, X_train.shape[2]), return_sequences=True))
    model_GRU.add(GRU(64, return_sequences=True))
    model_GRU.add(Dropout(rate=0.5))
    model_GRU.add(Flatten())
    model_GRU.add(Dense(512, activation="relu"))
    model_GRU.add(Dropout(rate=0.5))
    model_GRU.add(Dense(64, activation="relu"))
    model_GRU.add(Dense(1, activation='relu'))

    # Compile
    adam = optimizers.Adam(learning_rate=0.001)
    model_GRU.compile(loss="mse", optimizer=adam, metrics=["accuracy"])

    # Early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Train model
    history = model_GRU.fit(X_train, y_train, epochs=500, batch_size=64, validation_split=0.2, verbose=0, callbacks=[early_stopping])

    # Filter 2023 data
    data_23 = common_data[common_data['year'] == 2023]

    # Scale 2023 data
    X_2023_scaled = scaler_X.transform(data_23[features].values)

    # Convert to time series data format (use 2022, 2021, 2020 data to predict 2023)
    def create_sequences_for_prediction(X, seq_length):
        X_seq = []
        for i in range(len(X)):
            seq_x = X[max(0, i - seq_length + 1):i + 1]
            seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
            X_seq.append(seq_x)
        return np.array(X_seq)

    X_2023_seq = create_sequences_for_prediction(X_2023_scaled, seq_length)

    # Predict 2023 data
    y_pred_scaled = model_GRU.predict(X_2023_seq)

    # Restore scale
    y_pred = scaler_y.inverse_transform(y_pred_scaled)

    # Restore actual 2023 p_era values for comparison
    y_test_actual = data_23[target].values

    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_test_actual[:len(y_pred)], y_pred))
    rmse_list.append(rmse)

    # Calculate MAE
    mae = mean_absolute_error(y_test_actual[:len(y_pred)], y_pred)
    mae_list.append(mae)

    # Calculate MAPE
    mape = mean_absolute_percentage_error(y_test_actual[:len(y_pred)], y_pred)
    mape_list.append(mape)

    print(f'Iteration {i+1}/{iterations} - RMSE: {rmse}, MAE: {mae}, MAPE: {mape}')

# Calculate average RMSE, MAE, MAPE
avg_rmse = np.mean(rmse_list)
avg_mae = np.mean(mae_list)
avg_mape = np.mean(mape_list)

print(f'Average RMSE: {avg_rmse}')
print(f'Average MAE: {avg_mae}')
print(f'Average MAPE: {avg_mape}')

7/7 [==============================] - 2s 3ms/step
Iteration 1/5 - RMSE: 0.5776028636273925, MAE: 0.45864543818045345, MAPE: 0.13372412384799126
7/7 [==============================] - 1s 4ms/step
Iteration 2/5 - RMSE: 0.6152881710464299, MAE: 0.4938841767933057, MAPE: 0.1374930016468013
7/7 [==============================] - 2s 3ms/step
Iteration 3/5 - RMSE: 0.5597376973243573, MAE: 0.45045907458245465, MAPE: 0.1302327847380244
7/7 [==============================] - 1s 3ms/step
Iteration 4/5 - RMSE: 0.7067945181053076, MAE: 0.5754042684859123, MAPE: 0.1515705773617338
7/7 [==============================] - 2s 3ms/step
Iteration 5/5 - RMSE: 0.6905860731242487, MAE: 0.5490971256799744, MAPE: 0.15196961554435282
Average RMSE: 0.6300018646455472
Average MAE: 0.5054980167444201
Average MAPE: 0.14099802062778072


In [2]:
# Bi-GRU

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout, Flatten, Bidirectional
from tensorflow.keras import optimizers
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from tensorflow.keras.callbacks import EarlyStopping

# Load dataset
file_path = 'data/basic.csv'
data = pd.read_csv(file_path)

# Remove unnecessary columns
data = data.drop(columns=['last_name, first_name'])

# Handle missing values (e.g., replace with 0)
data = data.fillna(0)

# Extract player_id that exists in 2020, 2021, 2022, and 2023
data_2021 = data[data['year'] == 2021]
data_2022 = data[data['year'] == 2022]
data_2023 = data[data['year'] == 2023]

player_ids_2021 = set(data_2021['player_id'].unique())
player_ids_2022 = set(data_2022['player_id'].unique())
player_ids_2023 = set(data_2023['player_id'].unique())

common_player_ids = player_ids_2021 & player_ids_2022 & player_ids_2023

# Extract data corresponding to common player_id
common_data = data[data['player_id'].isin(common_player_ids)]

# Extract data corresponding to 2020, 2021, and 2022
final = common_data[common_data['year'].isin([2021, 2022])]
final = final.sort_values(by=['player_id', 'year'])

# Select necessary columns (excluding year)
features = [col for col in final.columns if col not in ['player_id', 'year', 'p_era']]
target = 'p_era'

# Separate independent and dependent variables
X = final[features].values
y = final[target].values

# Data scaling
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

# Convert to time series data
def create_sequences(X, y, seq_length):
    X_seq, y_seq = [], []
    for i in range(len(X)):
        seq_x = X[max(0, i - seq_length + 1):i + 1]
        seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
        seq_y = y[i]
        X_seq.append(seq_x)
        y_seq.append(seq_y)
    return np.array(X_seq), np.array(y_seq)

seq_length = 2  # Set sequence length
X_seq, y_seq = create_sequences(X_scaled, y_scaled, seq_length)

# Set training data same as entire data
X_train, y_train = X_seq, y_seq

# Initialize lists to store metrics
iterations = 5
rmse_list = []
mae_list = []
mape_list = []

for i in range(iterations):
    # Define Bidirectional GRU model
    model_BiGRU = Sequential()
    model_BiGRU.add(Bidirectional(GRU(64, return_sequences=True), input_shape=(seq_length, X_train.shape[2])))
    model_BiGRU.add(Bidirectional(GRU(64, return_sequences=True)))
    model_BiGRU.add(Dropout(rate=0.5))
    model_BiGRU.add(Flatten())
    model_BiGRU.add(Dense(512, activation="relu"))
    model_BiGRU.add(Dropout(rate=0.5))
    model_BiGRU.add(Dense(64, activation="relu"))
    model_BiGRU.add(Dense(1, activation='relu'))

    # Compile
    adam = optimizers.Adam(learning_rate=0.001)
    model_BiGRU.compile(loss="mse", optimizer=adam, metrics=["accuracy"])

    # Early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Train model
    history = model_BiGRU.fit(X_train, y_train, epochs=500, batch_size=64, validation_split=0.2, verbose=0, callbacks=[early_stopping])

    # Filter 2023 data
    data_23 = common_data[common_data['year'] == 2023]

    # Scale 2023 data
    X_2023_scaled = scaler_X.transform(data_23[features].values)
    
    def create_sequences_for_prediction(X, seq_length):
        X_seq = []
        for i in range(len(X)):
            seq_x = X[max(0, i - seq_length + 1):i + 1]
            seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
            X_seq.append(seq_x)
        return np.array(X_seq)

    # Convert to time series data (use 2022, 2021, 2020 data to predict 2023)
    X_2023_seq = create_sequences_for_prediction(X_2023_scaled, seq_length)

    # Predict 2023 data
    y_pred_scaled = model_BiGRU.predict(X_2023_seq)

    # Inverse scaling
    y_pred = scaler_y.inverse_transform(y_pred_scaled)

    # Restore actual 2023 p_era values for comparison
    y_test_actual = data_23[target].values

    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_test_actual[:len(y_pred)], y_pred))
    rmse_list.append(rmse)

    # Calculate MAE
    mae = mean_absolute_error(y_test_actual[:len(y_pred)], y_pred)
    mae_list.append(mae)

    # Calculate MAPE
    mape = mean_absolute_percentage_error(y_test_actual[:len(y_pred)], y_pred)
    mape_list.append(mape)

    print(f'Iteration {i+1}/{iterations} - RMSE: {rmse}, MAE: {mae}, MAPE: {mape}')

# Calculate average RMSE, MAE, MAPE
avg_rmse = np.mean(rmse_list)
avg_mae = np.mean(mae_list)
avg_mape = np.mean(mape_list)

print(f'Average RMSE: {avg_rmse}')
print(f'Average MAE: {avg_mae}')
print(f'Average MAPE: {avg_mape}')

7/7 [==============================] - 2s 3ms/step
Iteration 1/5 - RMSE: 0.6821283779163654, MAE: 0.5302002467740561, MAPE: 0.14744768163955235
7/7 [==============================] - 4s 4ms/step
Iteration 2/5 - RMSE: 0.6267220845736746, MAE: 0.5039213381642881, MAPE: 0.13752878082172879
7/7 [==============================] - 3s 4ms/step
Iteration 3/5 - RMSE: 0.6815939988469304, MAE: 0.5489307756009308, MAPE: 0.14951244730901428
7/7 [==============================] - 3s 4ms/step
Iteration 4/5 - RMSE: 0.5769444432589571, MAE: 0.4687038656594096, MAPE: 0.13371904287577577
7/7 [==============================] - 3s 4ms/step
Iteration 5/5 - RMSE: 0.6295517421492851, MAE: 0.5193267593752359, MAPE: 0.14255871659247274
Average RMSE: 0.6393881293490425
Average MAE: 0.5142165971147842
Average MAPE: 0.14215333384770878


In [3]:
# biattention GRU

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import GRU, Dense, Dropout, Flatten, Bidirectional, Input, Layer, dot, concatenate, Activation
from tensorflow.keras import optimizers
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras.backend as K

# Load dataset
file_path = 'data/basic.csv'
data = pd.read_csv(file_path)

# Remove unnecessary columns
data = data.drop(columns=['last_name, first_name'])

# Handle missing values (e.g., replace with 0)
data = data.fillna(0)

# Extract player_id that exists in 2020, 2021, 2022, and 2023

data_2021 = data[data['year'] == 2021]
data_2022 = data[data['year'] == 2022]
data_2023 = data[data['year'] == 2023]

player_ids_2021 = set(data_2021['player_id'].unique())
player_ids_2022 = set(data_2022['player_id'].unique())
player_ids_2023 = set(data_2023['player_id'].unique())

common_player_ids = player_ids_2021 & player_ids_2022 & player_ids_2023

# Extract data corresponding to common player_id
common_data = data[data['player_id'].isin(common_player_ids)]

# Extract data corresponding to 2020, 2021, and 2022
final = common_data[common_data['year'].isin([2021, 2022])]
final = final.sort_values(by=['player_id', 'year'])

# Select necessary columns (excluding year)
features = [col for col in final.columns if col not in ['player_id', 'year', 'p_era']]
target = 'p_era'

# Separate independent and dependent variables
X = final[features].values
y = final[target].values

# Data scaling
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

# Convert to time series data format
def create_sequences(X, y, seq_length):
    X_seq, y_seq = [], []
    for i in range(len(X)):
        seq_x = X[max(0, i - seq_length + 1):i + 1]
        seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
        seq_y = y[i]
        X_seq.append(seq_x)
        y_seq.append(seq_y)
    return np.array(X_seq), np.array(y_seq)

seq_length = 2  # Set sequence length
X_seq, y_seq = create_sequences(X_scaled, y_scaled, seq_length)

# Set training data same as entire data
X_train, y_train = X_seq, y_seq

# Define custom Attention Layer
class Attention(Layer):
    def __init__(self, **kwargs):
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], input_shape[-1]),
                                 initializer="glorot_uniform", trainable=True)
        self.b = self.add_weight(name="att_bias", shape=(input_shape[-1],),
                                 initializer="glorot_uniform", trainable=True)
        super(Attention, self).build(input_shape)

    def call(self, x):
        e = K.tanh(K.dot(x, self.W) + self.b)
        a = K.softmax(e, axis=1)
        output = x * a
        return K.sum(output, axis=1)

# Initialize lists to store metrics
iterations = 5
rmse_list = []
mae_list = []
mape_list = []

for i in range(iterations):
    # Define BiAttention GRU model
    input_seq = Input(shape=(seq_length, X_train.shape[2]))
    
    # Bidirectional GRU
    x = Bidirectional(GRU(64, return_sequences=True))(input_seq)
    x = Bidirectional(GRU(64, return_sequences=True))(x)
    
    # Apply Attention Layer
    attn_out = Attention()(x)
    
    # Fully connected layers
    x = Dense(512, activation="relu")(attn_out)
    x = Dropout(rate=0.5)(x)
    x = Dense(64, activation="relu")(x)
    output = Dense(1, activation='linear')(x)
    
    model_BiAttGRU = Model(inputs=input_seq, outputs=output)

    # Compile
    adam = optimizers.Adam(learning_rate=0.001)
    model_BiAttGRU.compile(loss="mse", optimizer=adam, metrics=["accuracy"])

    # Early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Train model
    history = model_BiAttGRU.fit(X_train, y_train, epochs=500, batch_size=64, validation_split=0.2, verbose=0, callbacks=[early_stopping])

    # Filter 2023 data
    data_23 = common_data[common_data['year'] == 2023]

    # Scale 2023 data
    X_2023_scaled = scaler_X.transform(data_23[features].values)
    
    def create_sequences_for_prediction(X, seq_length):
        X_seq = []
        for i in range(len(X)):
            seq_x = X[max(0, i - seq_length + 1):i + 1]
            seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
            X_seq.append(seq_x)
        return np.array(X_seq)

    # Convert to time series data format (use 2022, 2021, 2020 data to predict 2023)
    X_2023_seq = create_sequences_for_prediction(X_2023_scaled, seq_length)

    # Predict 2023 data
    y_pred_scaled = model_BiAttGRU.predict(X_2023_seq)

    # Restore scale
    y_pred = scaler_y.inverse_transform(y_pred_scaled)

    # Restore actual 2023 p_era values for comparison
    y_test_actual = data_23[target].values

    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_test_actual[:len(y_pred)], y_pred))
    rmse_list.append(rmse)

    # Calculate MAE
    mae = mean_absolute_error(y_test_actual[:len(y_pred)], y_pred)
    mae_list.append(mae)

    # Calculate MAPE
    mape = mean_absolute_percentage_error(y_test_actual[:len(y_pred)], y_pred)
    mape_list.append(mape)

    print(f'Iteration {i+1}/{iterations} - RMSE: {rmse}, MAE: {mae}, MAPE: {mape}')

# Calculate average RMSE, MAE, MAPE
avg_rmse = np.mean(rmse_list)
avg_mae = np.mean(mae_list)
avg_mape = np.mean(mape_list)

print(f'Average RMSE: {avg_rmse}')
print(f'Average MAE: {avg_mae}')
print(f'Average MAPE: {avg_mape}')

7/7 [==============================] - 3s 4ms/step
Iteration 1/5 - RMSE: 0.6281701094352593, MAE: 0.491720215525604, MAPE: 0.14715397958528637
7/7 [==============================] - 3s 4ms/step
Iteration 2/5 - RMSE: 0.5692956987393294, MAE: 0.4489616589384954, MAPE: 0.13227411153647847
7/7 [==============================] - 3s 4ms/step
Iteration 3/5 - RMSE: 0.5805149110707362, MAE: 0.4590998698893376, MAPE: 0.13222518080925422
7/7 [==============================] - 3s 4ms/step
Iteration 4/5 - RMSE: 0.5704166707448887, MAE: 0.4533328552522521, MAPE: 0.13504555135881652
7/7 [==============================] - 3s 4ms/step
Iteration 5/5 - RMSE: 0.5759041050813294, MAE: 0.46007183063433355, MAPE: 0.1328556946580225
Average RMSE: 0.5848602990143086
Average MAE: 0.4626372860480045
Average MAPE: 0.1359109035895716
